In [1]:
#!pip3 install tqdm
#!pip3 install git+https://git@github.com/SKTBrain/KoBERT.git@master
#!pip3 install seqeval
#!pip3 install kiwipiepy
#!pip3 install pymongo wikipedia koalanlp datasets thefuzz

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import argparse
from tqdm.notebook import tqdm
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
from data_loader import load_and_cache_examples
from utils import init_logger, load_tokenizer, set_seed, MODEL_CLASSES, MODEL_PATH_MAP
from trainer import Trainer

In [4]:
args = {
    "task": "naver-ner",
    "model_dir": "./model_timex3_kobert",
    "data_dir": "./data/modeldata/",
    "pred_dir": "./preds_timex3_kobert",
    "train_file": "timex3.train",
    "test_file": "timex3.test",
    "val_file": "timex3.val",
    "label_file": "label.timex3",
    "write_pred": True,
    "model_type": "kobert",
    "seed": 42,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "max_seq_len": 100,
    "learning_rate": 5e-5,
    "num_train_epochs": 40.0,
    "weight_decay": 0.0,
    "gradient_accumulation_steps": 1,
    "adam_epsilon": 1e-8,
    "max_grad_norm": 1.0,
    "max_steps": -1,
    "warmup_steps": 0,
    "logging_steps": 1000,
    "save_steps": 1000,
    "do_train": True,
    "do_eval": True,
    "no_cuda": False
}
args["model_name_or_path"] = MODEL_PATH_MAP[args["model_type"]]

In [5]:
tokenizer = load_tokenizer(args)

In [6]:
train_dataset = None
dev_dataset = None
test_dataset = None

if args["do_train"] or args["do_eval"]:
    test_dataset = load_and_cache_examples(args, tokenizer, mode="test", use_cache=False)
if args["do_train"]:
    train_dataset = load_and_cache_examples(args, tokenizer, mode="train", use_cache=False)
    dev_dataset = load_and_cache_examples(args, tokenizer, mode="dev", use_cache=False)

In [7]:
trainer = Trainer(args, train_dataset, dev_dataset, test_dataset)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/sdlee130/venv/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [8]:
if args["do_train"]:
    trainer.train()

Epoch:   5%|█▋                                | 2/40 [03:45<1:11:22, 112.71s/it]


Iteration:   0%|                                         | 0/46 [00:00<?, ?it/s]


Iteration:   2%|▋                                | 1/46 [00:02<01:51,  2.48s/it]


Iteration:   4%|█▍                               | 2/46 [00:05<01:53,  2.58s/it]


Iteration:   7%|██▏                              | 3/46 [00:07<01:45,  2.44s/it]


...

Iteration:  85%|███████████████████████████▏    | 39/46 [01:36<00:17,  2.49s/it]


Iteration:  87%|███████████████████████████▊    | 40/46 [01:38<00:14,  2.45s/it]


Iteration:  89%|████████████████████████████▌   | 41/46 [01:41<00:12,  2.51s/it]


Iteration:  91%|█████████████████████████████▏  | 42/46 [01:43<00:09,  2.47s/it]


Epoch: 100%|█████████████████████████████████| 40/40 [1:15:28<00:00, 113.22s/it]


In [9]:
trainer.save_model()

In [10]:
if args["do_eval"]:
    trainer.load_model()
    trainer.evaluate("test", "eval")

Evaluating: 100%|█████████████████████████████████| 7/7 [00:04<00:00,  1.49it/s]

***** Eval results *****
  %s = %s f1 0.5428571428571428
  %s = %s loss 0.21534028542893274
  %s = %s precision 0.5507246376811594
  %s = %s recall 0.5352112676056338
